In [25]:
import torch
from matplotlib.pyplot import xlabel, yscale, xscale
from torch import nn
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
import random
import collections
import re
import requests
import random 
from torch.nn import functional as F

In [50]:
with open('../data/time_machine.txt','r') as f:
    content=f.readlines()
##读取文章
def read_time_machine():
    with open('../data/time_machine.txt','r') as f:
        content=f.readlines()
    return [ re.sub('[^A-Za-z]', ' ', i.replace('\n','')).strip().lower() for i in content ]

## 定义一个拆分词元的函数 结果是词元组成的list
def tokenize(content,token='word'):
    if token=='word':
        token_list=[token.lower() for i in content for token in i.split(' ')]
    else:
        token_list=[token.lower() for i in content for token in i]
    return token_list

##定义一个统计频率的函数 可以处理1d2d
def count_corpus(token_list):
    if isinstance(token_list[0], list):
      tokens=[token for i in token_list for token in i ]
    tokens_count=collections.Counter(token_list)
    return tokens_count

class Vocal():
    def __init__(self,token_list=None,min_feq=0,reserved_tokens=None):
        self.token_list=token_list
        if token_list is None:
            self.token_list=[]
        if reserved_tokens is None:
            reserved_tokens=[]
        counter_info=count_corpus(token_list)
        self._token_feq=[]
        ##只接受符合条件的词
        for items in sorted(counter_info.items(),key=lambda x:x[1],reverse=True):
            if items[1]<=min_feq:
                break
            else:
                self._token_feq.append(items)
        
        ##  
        if '<unk>' in reserved_tokens:
            self.idx_to_token=reserved_tokens
        else:
            self.idx_to_token=['unk']+reserved_tokens
            
        self.token_to_idx={token:i for i,token in enumerate(self.idx_to_token)}
        for token,_ in self._token_feq:
            self.idx_to_token.append(token)
            self.token_to_idx[token]=len(self.token_to_idx)
    def __len__(self):
        return len(self.token_to_idx)
    ##实现一个索引方法 但是传入的索引是token 返回是idx,保证未知token显示0值
    def __getitem__(self,tokens):
        if not isinstance(tokens,(tuple,list)):
            return self.token_to_idx.get(tokens,0)
        return [self.__getitem__(i) for i in tokens]
    
    @property
    def unk(self):
        return 0
    
    @property
    def tokens_freq(self):
        return self._token_feq
        
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    tokens=tokenize(read_time_machine())
    vocal=Vocal(tokens,'char')

    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocal[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

In [45]:
load_corpus_time_machine

In [46]:
tokens=tokenize(read_time_machine(),'char')
vocal=Vocal(tokens)

In [48]:
corpus=[vocal[token] for token in tokens]

In [49]:
corpus

[3,
 9,
 2,
 1,
 3,
 5,
 13,
 2,
 1,
 13,
 4,
 15,
 9,
 5,
 6,
 2,
 1,
 1,
 21,
 19,
 1,
 9,
 1,
 1,
 18,
 1,
 1,
 17,
 2,
 12,
 12,
 8,
 5,
 3,
 9,
 2,
 1,
 3,
 5,
 13,
 2,
 1,
 3,
 10,
 4,
 22,
 2,
 12,
 12,
 2,
 10,
 1,
 1,
 16,
 7,
 10,
 1,
 8,
 7,
 1,
 5,
 3,
 1,
 17,
 5,
 12,
 12,
 1,
 21,
 2,
 1,
 15,
 7,
 6,
 22,
 2,
 6,
 5,
 2,
 6,
 3,
 1,
 3,
 7,
 1,
 8,
 20,
 2,
 4,
 23,
 1,
 7,
 16,
 1,
 9,
 5,
 13,
 17,
 4,
 8,
 1,
 2,
 24,
 20,
 7,
 14,
 6,
 11,
 5,
 6,
 18,
 1,
 4,
 1,
 10,
 2,
 15,
 7,
 6,
 11,
 5,
 3,
 2,
 1,
 13,
 4,
 3,
 3,
 2,
 10,
 1,
 3,
 7,
 1,
 14,
 8,
 1,
 1,
 9,
 5,
 8,
 1,
 18,
 10,
 2,
 19,
 1,
 2,
 19,
 2,
 8,
 1,
 8,
 9,
 7,
 6,
 2,
 1,
 4,
 6,
 11,
 3,
 17,
 5,
 6,
 23,
 12,
 2,
 11,
 1,
 1,
 4,
 6,
 11,
 1,
 9,
 5,
 8,
 1,
 14,
 8,
 14,
 4,
 12,
 12,
 19,
 1,
 20,
 4,
 12,
 2,
 1,
 16,
 4,
 15,
 2,
 1,
 17,
 4,
 8,
 1,
 16,
 12,
 14,
 8,
 9,
 2,
 11,
 1,
 4,
 6,
 11,
 1,
 4,
 6,
 5,
 13,
 4,
 3,
 2,
 11,
 1,
 1,
 3,
 9,
 2,
 16,
 5,
 10,
 2,
 1,
 21,
 14

4581

In [ ]:


batch_size, num_steps = 32, 35
train_iter, vocab = load_data_time_machine(batch_size, num_steps)